In [1]:
import os
import numpy as np
import rasterio
import matplotlib.image
import matplotlib.pyplot as plt

from pathlib import Path


In [2]:
files = {}

for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[1][:6] == "202203":
            if parts[0] not in files:
                files[parts[0]] = {}
            if parts[1] not in files[parts[0]]:
                files[parts[0]][parts[1]] = {}
            files[parts[0]][parts[1]][parts[2]] = f"data/{f}"


In [3]:
files


{'T35VNE': {'20220322T093029': {'B02': 'data/T35VNE_20220322T093029_B02_10m.jp2',
   'B03': 'data/T35VNE_20220322T093029_B03_10m.jp2',
   'B08': 'data/T35VNE_20220322T093029_B08_10m.jp2',
   'SCL': 'data/T35VNE_20220322T093029_SCL_20m.jp2',
   'B04': 'data/T35VNE_20220322T093029_B04_10m.jp2'},
  '20220320T094031': {'B02': 'data/T35VNE_20220320T094031_B02_10m.jp2',
   'B04': 'data/T35VNE_20220320T094031_B04_10m.jp2',
   'B08': 'data/T35VNE_20220320T094031_B08_10m.jp2',
   'SCL': 'data/T35VNE_20220320T094031_SCL_20m.jp2',
   'B03': 'data/T35VNE_20220320T094031_B03_10m.jp2'},
  '20220314T092031': {'B02': 'data/T35VNE_20220314T092031_B02_10m.jp2',
   'B03': 'data/T35VNE_20220314T092031_B03_10m.jp2',
   'B04': 'data/T35VNE_20220314T092031_B04_10m.jp2',
   'B08': 'data/T35VNE_20220314T092031_B08_10m.jp2',
   'SCL': 'data/T35VNE_20220314T092031_SCL_20m.jp2'}},
 'T35VND': {'20220322T093029': {'B02': 'data/T35VND_20220322T093029_B02_10m.jp2',
   'B03': 'data/T35VND_20220322T093029_B03_10m.jp2',

In [4]:
c = 0

offsets = {
    'T34UDG': (30000, 0),
    'T34UEG': (30000, 10000),
    'T34UFG': (30000, 20000),
    'T34VDH': (20000, 0),
    'T34VDJ': (10000, 0),
    'T34VDK': (0, 0), # top left
    'T34VEH': (20000, 10000),
    'T34VEJ': (10000, 10000),
    'T34VEK': (0+100, 10000),
    'T34VFH': (20000, 20000),
    'T34VFJ': (10000, 20000),
    'T34VFK': (0, 20000),
    'T35ULB': (30000+100, 30980-c),
    'T35UMB': (30000+100, 40980-c),
    'T35UNB': (30000+100, 50980-c),
    'T35VLC': (20000+100, 30980-c),
    'T35VLD': (10000+100, 30980-c),
    'T35VLE': (0+100, 30980-c),
    'T35VMC': (20000+100, 40980-c),
    'T35VMD': (10000+100, 40980-c),
    'T35VME': (0+100, 40980-c),
    'T35VNC': (20000+100, 50980-c),
    'T35VND': (10000+100, 50980-c),
    'T35VNE': (0+100, 50980-c)
}


In [5]:
rgb_composite = np.dstack(
    (np.full((21180, 47410), 0.0), np.full((21180, 47410), 0.0), np.full((21180, 47410), 0.0))
)


In [6]:
cells = ["T34VFK", "T34VFJ", "T35VLE", "T35VLD"]


In [7]:
dates = []
for c in cells:
    dates += list(files[c])
dates = sorted(list(set(dates)))
    

In [8]:
files = {k:v for k,v in files.items() if k in cells}


In [9]:
gain = 2

for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    #print("reading green", fj["B03"])
                    c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)             
                except BaseException as e:
                    if os.path.isfile(fj["B03"]):
                        print("REMOVED", fj["B03"])
                        os.remove(fj["B03"])
                    raise e
                try:
                    #print("reading blue", fj["B02"])
                    c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)              
                except BaseException as e:
                    if os.path.isfile(fj["B02"]):
                        print("REMOVED", fj["B02"])
                        os.remove(fj["B02"])
                    raise e

                mask = ((scl != 0) & (scl != 1))
                #print(mask)
                #red[mask] = c_red[mask]
                #green[mask] = c_green[mask]
                #blue[mask] = c_blue[mask]
                #print("applying red")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 0][mask] = c_red[mask]
                #print("applying green")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = c_green[mask]
                #print("applying blue")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 2][mask] = c_green[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
        else:
            print("no match", ki, d)
    print("saving composite")
    matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/experiment/{d}.jpeg", rgb_composite)


20220301T100029
matched T34VFJ 20220301T100029
no match T35VLD 20220301T100029
matched T34VFK 20220301T100029
no match T35VLE 20220301T100029
saving composite
20220316T100031
matched T34VFJ 20220316T100031
no match T35VLD 20220316T100031
matched T34VFK 20220316T100031
no match T35VLE 20220316T100031
saving composite
20220318T095029
no match T34VFJ 20220318T095029
no match T35VLD 20220318T095029
no match T34VFK 20220318T095029
matched T35VLE 20220318T095029
saving composite
20220320T094031
matched T34VFJ 20220320T094031
ERROR: T34VFJ 20220320T094031 failed with 'B03'
matched T35VLD 20220320T094031
matched T34VFK 20220320T094031
matched T35VLE 20220320T094031
saving composite
20220321T100029
matched T34VFJ 20220321T100029
no match T35VLD 20220321T100029
matched T34VFK 20220321T100029
no match T35VLE 20220321T100029
saving composite
20220322T093029
no match T34VFJ 20220322T093029
matched T35VLD 20220322T093029
no match T34VFK 20220322T093029
no match T35VLE 20220322T093029
saving composit